# Projeto 2 - Predição de preços de casas

Peça a um comprador de imóveis para descrever a casa dos seus sonhos e eles provavelmente não começarão com a altura do teto do porão ou a proximidade de uma linha de trem. Mas este conjunto de dados prova que há muitos detalhes que influenciam as negociações de preços do que o número de quartos ou uma cerca branca.

# Sobre este projeto
Os principais pontos que serão avaliados:
*   Levantamento de hipóteses
*   Criação e seleção de características
*   Manipulação de dados e criação de gráficos simples com o Pandas
*   Criar um modelo de predição
*   Apresentação dos resultados

# Preparação do ambiente
Para este projeto, acessem o link - https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques e logo abaixo cliquem em "Download". Caso voce não tenha uma conta no Kaggle, crie uma e retorne para esse ponto para realizar o download. Descompacte o arquivo.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, SGDRegressor, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, max_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.feature_selection import f_classif, SelectKBest, VarianceThreshold

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
df = df.drop('Id', axis = 1)

# Exercício
Neste projeto você deverá desenvolver a partir do que já vimos nos projetos anteriores, ou seja, poderá usar qualquer um dos modelos vistos durante as aulas. Siga este roteiro e preencha cada parte.

# 1. Análise exploratória de dados (1.0 ponto)
Realize a análise exploratória de dados. Observe cada variável, como se comportam em relação a outras variáveis independentes e com a variável dependente.

In [ ]:
df.shape

(1460, 80)

In [ ]:
# Separação das variáveis em númericas e categóricas
df_categoricas = df.select_dtypes('object')
df_numericas = df.select_dtypes(exclude='object')

# Dentro do dataframe com variáveis numéricas, ainda temos as variáveis categoricas ordinais presentes no dataset
# As variáveis 'MSSubClass', 'OverallQual', 'OverallCond' precisam ser adicionadas ao dataframe de categoricas
df_categoricas = pd.concat([df_categoricas, df_numericas[['MSSubClass', 'OverallQual', 'OverallCond']]], axis = 1)
df_numericas = df_numericas.drop(['MSSubClass', 'OverallQual', 'OverallCond'], axis = 1)

# A variável target é inserida no df de variáveis categoricas para facilitar a construção dos gráficos, será excluida posteriormente.
df_categoricas = pd.concat([df_categoricas,df_numericas['SalePrice']], axis=1)

print(df_categoricas.shape)
print(df_numericas.shape)

(1460, 47)
(1460, 34)


In [ ]:
plt.figure(figsize=(30,24))
sns.heatmap(df_numericas.corr(), vmin=-1, vmax=1, annot=True);

Analisando a matriz de correlações, percebemos que há algumas variáveis com um alto nível de correlação com a variável 'SalePrice'. Quando analisamos as relações entre as variáveis independentes percebemos algumas relações.  
Há algumas correlações negativas significativas em relação entre as variáveis, porém, nenhuma em relação a 'SalePrice'. Para simplificar a análise, a matriz de correlações foi filtrada para exibir somente correlações acima de 0.5 ou abaixo de -0.3.


In [ ]:
correlacoes = df_numericas.corr().values
np.fill_diagonal(correlacoes, np.nan)
correlacoes = pd.DataFrame(correlacoes, columns = df_numericas.corr().columns, index = df_numericas.corr().index)

plt.figure(figsize=(10,10))
sns.heatmap(correlacoes[(correlacoes > 0.5) | (correlacoes < -0.3)].dropna(how='all').dropna(how='all', axis = 1), vmin=-1, vmax=1, annot=True);

In [ ]:
# Cruzamento das variáveis com correlação > 0.5 e <-0.3
vetor_corr = [] # Vetor a ser utilizado posteriormente na criação de gráficos

for i in range(len(correlacoes.columns)):
  for j in range(i,len(correlacoes.index)): # A matriz é quadrada e simétrica, evita que comparações descenecessarias sejam feitas analisando apenas a metade inferior da matriz.
    if correlacoes.iloc[i,j] == 1:
      continue
    
    if correlacoes.iloc[i,j] > 0.5 or correlacoes.iloc[i,j] < -0.3:
      print(f'{correlacoes.columns[i]} x {correlacoes.index[j]}')
      vetor_corr.append(f'{correlacoes.columns[i]} {correlacoes.index[j]}')

YearBuilt x YearRemodAdd
YearBuilt x GarageYrBlt
YearBuilt x GarageCars
YearBuilt x EnclosedPorch
YearBuilt x SalePrice
YearRemodAdd x GarageYrBlt
YearRemodAdd x SalePrice
BsmtFinSF1 x BsmtUnfSF
BsmtFinSF1 x TotalBsmtSF
BsmtFinSF1 x BsmtFullBath
BsmtUnfSF x BsmtFullBath
TotalBsmtSF x 1stFlrSF
TotalBsmtSF x SalePrice
1stFlrSF x GrLivArea
1stFlrSF x SalePrice
2ndFlrSF x GrLivArea
2ndFlrSF x HalfBath
2ndFlrSF x BedroomAbvGr
2ndFlrSF x TotRmsAbvGrd
GrLivArea x FullBath
GrLivArea x BedroomAbvGr
GrLivArea x TotRmsAbvGrd
GrLivArea x SalePrice
FullBath x TotRmsAbvGrd
FullBath x SalePrice
BedroomAbvGr x TotRmsAbvGrd
TotRmsAbvGrd x SalePrice
GarageYrBlt x GarageCars
GarageYrBlt x GarageArea
GarageCars x GarageArea
GarageCars x SalePrice
GarageArea x SalePrice


Podemos levantar algumas hipóteses baseadas nessas correlações. Como por exemplo:

*   'GarageYrBlt' x 'YearBuilt': 'GarageYrBlt' representa o ano que a garagem foi construída. 'YearBuilt' representa o ano da construção. A correlação entre as variáveis faz sentido uma vez que garagens geralmente são construídas junto com a casa.
*   'GarageYrBlt' x 'YearRemodAdd': 'GarageYrBlt' representa o ano que a garagem foi construída. 'YearRemodAdd' é a data de remodelação da casa. Essa correlação faz sentido uma vez que garagens podem ser adicionadas junto a remodelagem da casa.
*   'GarageCars' x 'YearBuilt': 'GarageCars' representa o tamanho da garagem em capacidade de carros. 'YearBuilt' representa o ano da construção. Provavelmente uma correlação espúria, talvez tenha alguma relação com o passar dos anos e na popularização de compras de carros.
*   'EnclosedPorch' x 'YearBuilt': 'EnclosedPorch' representa o tamanho da varanda em pés quadrados. 'YearBuilt' representa o ano da construção. Mostra uma relação negativa entre a área da varanda e o ano de construção, mostra que com o passar dos anos as varandas tendem a ficar menores, a popularização de apartamentos ou casas menores podem justificar essa relação.

Porém, as relações mais importantes são as que tem muita correlação com a variável target 'SalePrice', sendo assim, a análise será aprofundada nessa variável.


In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(data=df_numericas.corr().drop('SalePrice')['SalePrice'].reset_index().sort_values(by='SalePrice', ascending=False), y ='index', x = 'SalePrice', orient='h');

Observamos que as variáveis numéricas que possuem correlação maior que 0.5 com 'SalePrice' são: 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFloorSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt' e 'YearRemodAdd'.

Para a análise das variáveis categóricas, primeiramente precisamos discretizar a faixa 'SalesPrice' em faixas para que a visualização fique mais clara.


In [ ]:
rotulos = pd.cut(x=df_categoricas['SalePrice'], bins=[0,100000,200000,300000,400000,500000,600000,700000,800000]) # Discretiza os valores de 'SalesPrice em faixas de 100000 para rotular os gráficos
rotulos = rotulos.astype('str').str.replace(",", " até").str.replace("(", "").str.replace("]", "") # Organiza os rotulos para ficarem legiveis

plt.subplots(12,4,figsize=(7*3,7*10))
for i in range(0,len(df_categoricas.columns)):
  if df_categoricas.columns[i] == 'SalePrice':
    continue
  plt.subplot(12,4,i+1)
  sns.histplot(data=df_categoricas, x = df_categoricas.columns[i], hue=rotulos, hue_order = np.sort(rotulos.unique()), multiple='stack' )
  plt.xticks(rotation=90)
  plt.title(df_categoricas.columns[i])
  plt.xlabel('')

plt.tight_layout()
plt.show()

Analisando os histogramas das variáveis categóricas, percebemos algumas caracteristicas que tornam a analise dessas variáveis ainda mais complexa. A maioria das observações possuem 'SalePrice' entre 100000 até 200000, além de termos vários atributos categóricos com um unico valor dominante (por exemplo, variável 'Street' possui em sua grande maioria o valor 'Pave').

Não há muito oque se fazer em relação aos valores dominantes dentro dos atributos, porém, para melhorar a visualização, podemos binarizar o esquema de cores para trazer mais clareza ao gráfico.

In [ ]:
rotulos = rotulos.map({'100000 até 200000': '100000 até 200000',
                       '200000 até 300000': 'Outras faixas',
                       '300000 até 400000': 'Outras faixas',
                       '0 até 100000': 'Outras faixas',
                       '400000 até 500000': 'Outras faixas',
                       '500000 até 600000': 'Outras faixas',
                       '700000 até 800000': 'Outras faixas', 
                       '600000 até 700000': 'Outras faixas'})

In [ ]:
plt.subplots(12,4,figsize=(7*3,7*10))
for i in range(0,len(df_categoricas.columns)):
  if df_categoricas.columns[i] == 'SalePrice':
    continue
  plt.subplot(12,4,i+1)
  sns.histplot(data=df_categoricas, x = df_categoricas.columns[i], hue=rotulos, hue_order =  np.sort(rotulos.unique()), multiple='stack' )
  plt.xticks(rotation=90)
  plt.title(df_categoricas.columns[i])
  plt.xlabel('')

plt.tight_layout()
plt.show()

Agora, sabendo que a maioria das casas pertence a faixa de preço de 100000 a 200000, podemos analisar quais são os valores das variáveis categóricas que fazem com que o preço de venda fuja deste padrão (seja para cima ou para baixo).
A variável 'LotShape', por exemplo, ilustra bem a situação. 

*    Em terrenos regulares ('Reg'), temos o padrão de preço esperado, este padrão é o mesmo observado entre valores dominantes de outras variáveis categóricas dentro do dataset.
*   Quando analisamos terrenos levemente irregulares ('IR1') há uma mudança na proporção entre casas vendidas na faixa de preço padrão e em outras faixas.
*   O mesmo ocorre com os outros valores de 'Reg'. A proporção entre preços em 'IR2' e 'IR3' passa a ser dominado por 'Outras Faixas'.

Outra variável que podemos analisar é a variável 'Neighborgood', vemos claramente que a mudança de vizinhança faz com que a faixa de preço se desloque da faixa de preço mais comum no dataset, em especial o valor 'NridgHt' apresenta valores predominantemente fora da faixa de '100000 até 200000'.


In [ ]:
plt.subplots(12,4,figsize=(7*3,7*10))
for i in range(0,len(df_categoricas.columns)):
  if df_categoricas.columns[i] == 'SalePrice':
    continue
  plt.subplot(12,4,i+1)
  sns.boxplot(data=df_categoricas, x = df_categoricas.columns[i], y = df_categoricas['SalePrice'])
  plt.xticks(rotation=90)
  plt.title(df_categoricas.columns[i])
  plt.xlabel('')

plt.tight_layout()
plt.show()

Através dos boxplot, podemos realizar a análise de um ponto de vista puramente numérico. Aqui, é possível comprovar a hipótese de que a maior parte das vendas foram fechadas na faixa de 100000 até 200000 observando que o 1Q a mediana e o 3Q (ou seja, 50% dos dados) estão inseridos justamente nessa faixa.

Também podemos notar relações que não são possíveis em um histograma. A variável 'Condition2', por exemplo, mostra que casas próximas a parques ou áreas verdes tendem a ser bem valorizadas em contrapartida casas posicionadas próximas a vias arteriais (uma via arterial é uma estrada urbana de alta capacidade) tendem a desvalorizar a casa.

A relação mais notável neste gráfico é do aumento contínuo de 'SalePrice' junto a variável 'OverallQual', sendo uma variável que julga a qualidade dos materiais utilizados na construção, faz sentido que as casas sejam mais caras quanto melhor forem os materiais.


# 2. Levantamento de hipóteses (1.0 ponto)
Descreva quais hipóteses você observou ao fazer a análise exploratória de dados.

Da matriz de correlações, percebemos que há algumas variáveis com um alto nível de correlação entre elas e com a variável target. Nota-se que muitas das variáveis numéricas possuem correlação entre si que inicialmente parecem ser correlações espúrias, porém, existem variáveis que realmente apresentam relações de causa e dependência. Por exemplo:  
*  'YearBuilt' x 'YearRemodAdd': Representa o ano de construção e o ano da remodelagem, não há uma relação clara de causa e dependência. 
*  'YearBuilt' x 'GarageYrBlt': Essa aparenta ser uma relação de causa e dependência já que garagens geralmente são construídas junto com a casa. 
*  'YearBuilt' x 'GarageCars': 'GarageCars' representa o tamanho da garagem em capacidade de carros. Uma correlação espúria, não há causa e dependência aparente.
*  'YearBuilt' x 'EnclosedPorch': 'EnclosedPorch' representa o tamanho da varanda em pés quadrados. 'YearBuilt' representa o ano da construção. Não apresenta uma relação de causa e consequência
*  'YearRemodAdd' x 'GarageYrBlt': Outra relação que aparenta ser uma relação de causa e dependência já que garagens podem ser adicionadas junto a remodelagem da casa
*  'BsmtFinSF1' x 'BsmtUnfSF': ‘BsmtFinSF1’ é a área finalizada do porão tipo 1 (parcialmente finalizado). ‘BsmtUnfSF’ é a área não finalizada do porão. Apesar de estarem relacionadas ao porão, são tipos diferentes de finalização presentes nas casas do EUA, então não há causa e dependência.
*  'BsmtFinSF1' x 'TotalBsmtSF': ‘TotalBsmtSF’ trata da área total do porão, independente do tipo de finalização. A combinação dos tipos de área de finalizadas formam a área total do porão. São dependentes e uma causa a outra.
*  'BsmtFinSF1' x 'BsmtFullBath': ‘BsmtFullBath’ trata dos banheiros completos no porão. Para haver banheiros no porão, é necessário haver área, porém não é uma relação de causa e dependência.
*  'BsmtUnfSF' x 'BsmtFullBath': A mesma relação descrita anteriormente se encaixa aqui. Para haver banheiros no porão, é necessário haver área, porém não é uma relação de causa e dependência.
*  'TotalBsmtSF' x '1stFlrSF': ‘1stFlrSF’ trata da área do primeiro andar. Não apresenta causa e dependência em relação a área total do porão.
*  '1stFlrSF' x 'GrLivArea': ‘GrLivArea’ é a área construída acima do nível da terra. Outra variável que depende de outras, essa variável é a soma de ‘1stFlrSF’ e ‘2ndFlrSF’, então, são dependentes.
*  '2ndFlrSF' x 'GrLivArea': ‘2ndFlrSF’ é a área do segundo andar. A mesma relação descrita acima. A variável depende de outras, essa variável é a soma de ‘1stFlrSF’ e ‘2ndFlrSF’, então, são dependentes.
*  '2ndFlrSF' x 'HalfBath': ‘HalfBath’ trata do número de ‘meio banheiro’ (banheiros sem chuveiro, apenas privada e lavatório). Não possuem relação de causa e dependência.
*  '2ndFlrSF' x 'BedroomAbvGrd': ‘BedroomAbvGrd’ é o número de quartos acima do nível da terra. Não tem causa e dependência com com a área do segundo andar.
*  '2ndFlrSF' x 'TotRmsAbvGrd': ‘TotRmsAbvGrd’ número total de salas acima do nível da terra. Novamente uma correlação espúria, não há causalidade ou dependência.
*  'GrLivArea' x 'FullBath': ‘FullBath’ é o número de banheiros completos acima do nível da terra. Não há relação de causa e dependência com a área da casa.
*  'GrLivArea' x 'BedroomAbvGr': Não é uma relação de causa e dependência (área da casa x nº de quartos)
*  'GrLivArea' x 'TotRmsAbvGrd': Não é uma relação de causa e dependência (área da casa x nº de salas)
*  'FullBath' x 'TotRmsAbvGrd': Banheiros completos e salas acima do nível da terra. Não há uma relação de causa e dependência.
*  'BedroomAbvGr' x 'TotRmsAbvGrd': Número de quartos acima do nível da terra e número de salas acima do nível da terra. Não há uma relação de causa e dependência entre as variáveis.
*  'GarageYrBlt' x 'GarageCars': Ano de construção da garagem e número de carros. Como discutido anteriormente, não é uma relação de dependência.
*  'GarageYrBlt' x 'GarageArea': Ano de construção da garagem e área da garagem. Não é uma relação de dependência. 
*  'GarageCars' x 'GarageArea': ‘GarageCars’ é a capacidade da garagem em carros. Ambas as variáveis tratam da capacidade da garagem e são dependentes.


Ao analisar mais profundamente as correlações com a variável target, observamos que as variáveis numéricas de maior destaque são: 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFloorSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt' e 'YearRemodAdd'. 

Quanto as variáveis categóricas, percebemos que há diversos valores dominantes dentro das variáveis, ou seja, existe um padrão bem definido em relação as variáveis categóricas, a maioria das casas possuem características semelhantes. Também notamos que a maioria das casas pertence a faixa de preço entre 100000 até 200000 e que quando as características dessas casas fogem do padrão estabelecido pelas características dominantes, temos uma mudança nos preços. Utilizando a variável ‘LotShape’ como exemplo:

*    Em terrenos regulares ('Reg'), temos o padrão de preço esperado, este padrão é o mesmo observado entre valores dominantes de outras variáveis categóricas dentro do dataset.
*   Quando analisamos terrenos levemente irregulares ('IR1') há uma mudança na proporção entre casas vendidas na faixa de preço padrão e em outras faixas.
*   O mesmo ocorre com os outros valores de 'Reg'. A proporção entre preços em 'IR2' e 'IR3' passa a ser dominado por 'Outras Faixas'.

Esse mesmo efeito acontece com outras variáveis, como por exemplo: ‘OverallCond’, ‘OverallQual’, ‘SaleType’, 'KitchenQual'. 

Também é importante notar que as variáveis categóricas abordam diversas características das casas e casas que não possuem essas características não são abordadas nos gráficos, a exemplo da variável ‘PoolQC’ onde há apenas 7 valores, isso significa que a maior parte do dataset são de casas sem piscinas e que possuem o mesmo perfil de preço das variáveis anteriores.  

Através dos boxplot, podemos realizar a análise de um ponto de vista puramente numérico, então, podemos notar relações que não são possíveis em um histograma. A variável 'Condition2', por exemplo, mostra que casas próximas a parques ou áreas verdes tendem a ser bem valorizadas em contrapartida casas posicionadas próximas a vias arteriais (uma via arterial é uma estrada urbana de alta capacidade) tendem a desvalorizar a casa.  

A hipótese mais notável é do aumento contínuo de 'SalePrice' junto a variável 'OverallQual', sendo uma variável que julga a qualidade dos materiais utilizados na construção, faz sentido que as casas sejam mais caras quanto melhor forem os materiais.



# 3. Apresentação das ideias obtidas (1.0 ponto)
Apresente com gráficos as suas observações e a descreva cada gráfico.

In [ ]:
# Regplots irão mostrar as relações numericas.
# Violinplots e histogramas irão mostrar as relações das variaveis categoricas com a target. E como sair de seus valores mais frequentes modifica o preço médio de venda.

df_graficos_num = df_numericas[correlacoes.columns]
df_graficos_cat = df_categoricas[['OverallCond', 'OverallQual', 'SaleType','PoolQC','LotShape','KitchenQual', 'SalePrice']]
df_graficos_cat['PoolQC'] = df_graficos_cat['PoolQC'].fillna('Sem piscina')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
plt.subplots(8,4,figsize=(4*5,8*4))
for i in range(len(vetor_corr)):

  plt.subplot(8,4,i+1)
  variaveis = vetor_corr[i].split()
  sns.regplot(data=df_graficos_num, x=variaveis[0], y=variaveis[1], marker="+")
  plt.title(f'{variaveis[0]} x {variaveis[1]}')

plt.tight_layout()
plt.show()

Os gráficos mostram as correlações entre as variáveis para visualização das hipóteses explanadas anteriormente. Uma regressão também é feita para se avaliar o tipo de correlação entre ambas. O objetivo nesta análise é de identificar correlações espúrias e relações de causalidade e dependência.
*  'YearBuilt' x 'YearRemodAdd': O gráfico mostra a relação de dependência entre as duas variáveis. As casas mais recentes, geralmente não passam por remodelagem e então passam a ter o mesmo valor em ambas as variáveis (formando uma reta). As casas mais antigas também passam pelo processo de remodelagem, tornando a variável ‘YearBuild’ desnecessária. 
*  'YearBuilt' x 'GarageYrBlt': Quando casas são construídas, geralmente garagens também são construídas. Dessa forma, essas variáveis possuem uma relação de dependência, formando uma reta que se aproxima da linha de regressão. Dessa forma, a variável 'GarageYrBlt' também é desnecessária.
*  'YearBuilt' x 'GarageCars': 'GarageCars' é uma variável discreta, quando plotada com um regplot é possível ver o seu crescimento ao passar do tempo. Apesar de haver correlação, não é possui uma relação de causa ou dependência.
*  'YearBuilt' x 'EnclosedPorch': De forma similar, é possível notar a diminuição da área das varandas com o passar o tempo, com muitas situações iguais a 0 a partir de 1940. Apesar de haver correlação, não há relação de causalidade ou dependência.
*  'YearBuilt' x SalePrice: A relação de ‘YearBuilt’ e ‘SalePrice’. Notamos a alta correlação citada anteriormente nas hipóteses.
*  'YearRemodAdd' x 'GarageYrBlt': Assim como na análise de ‘YearBuilt’, as garagens são adicionadas na construção da casa ou na remodelagem, é uma situação de dependência.
*  'YearRemodAdd' x ‘SalePrice’: Relação entre o ano de remodelagem da casa. Percebemos uma correlação positiva.
*  'BsmtFinSF1' x 'BsmtUnfSF': A correlação entre os tipos de área do porão. De forma lógica, se possui um tipo de área, a tendencia é possuir menos de outro tipo. Como se trata de tipos de área, não há causalidade ou dependência.
*  'BsmtFinSF1' x 'TotalBsmtSF': Outra correlação entre os tipos de área do porão. Da mesma forma, se a casa possui um tipo de área, a tendencia é possuir menos de outro tipo. Como se trata de tipos de área, não há causalidade ou dependência.
*  'BsmtFinSF1' x 'BsmtFullBath': A relação entre os banheiros e área do tipo 1 do porão. Quanto mais banheiros mais área, porém, sem causalidade.
*  'BsmtUnfSF' x 'BsmtFullBath': Relação parecida com a anterior, porém, trata-se de um tipo específico de porão (não finalizado). Se tratando de porões não finalizados, faz sentido que quanto menos área finalizada, menos banheiros. Não há dependência. 
*  'TotalBsmtSF' x '1stFlrSF': Relação entre área do primeiro andar e área do porão. Por se tratar de áreas e uma estar acima da outra, faz sentido que tenham valores parecidos, porém sem dependência.
*  'TotalBsmtSF' x '1stFlrSF': Relação entre a área do porão e o preço de venda da casa. Vemos uma correlação positiva.
*  '1stFlrSF' x 'GrLivArea': Relação entre área da casa e área do primeiro andar. A variável 'GrLivArea' se trata da soma de ‘1stFlrSF’ e ‘2ndFlrSF’ e quando ‘2ndFlrSF’ é igual a 0, 'GrLivArea' se torna igual a ‘1stFlrSF’. Essas variáveis possuem dependência.
*  '1stFlrSF' x 'SalePrice': Relação linear positiva em relação a variável target. 
*  '2ndFlrSF' x 'GrLivArea': Assim como '1stFlrSF' x 'GrLivArea', a relação de dependência fica evidenciada devido a soma das partes serem iguais a 'GrLivArea'.
*  '2ndFlrSF' x 'HalfBath': O número de meio banheiros em relação a área do segundo andar, não possuem relação de causa e dependência.
*  '2ndFlrSF' x 'BedroomAbvGrd': De forma similar, o número de quartos não tem relação de causa ou dependência com a área do segundo andar.
*  '2ndFlrSF' x 'TotRmsAbvGrd': De forma similar, o número de salas não tem relação de dependência ou causalidade com a área do segundo andar.
*  'GrLivArea' x 'FullBath': Area total em relação ao número de banheiros completos. Sem relação de dependencia.
*  'GrLivArea' x 'BedroomAbvGr': Area total em relação ao número de quartos. Sem relação de dependência.
*  'GrLivArea' x 'TotRmsAbvGrd': Area total em relação ao número de salas. Sem relação de dependência.
*  'GrLivArea' x 'SalePrice': Relação linear positiva entre preço e área total da casa.
PAREI AQUI
*  'FullBath' x 'TotRmsAbvGrd': Banheiros completos e salas acima do nível da terra. Não há uma relação de causa e dependência.
*  'FullBath' x 'SalePrice': Relação linear positiva entre o número de banheiros completos e a variável target.
*  'BedroomAbvGr' x 'TotRmsAbvGrd': Relação linear positive entre número de banheiros completos e numero de salas acima do nível da terra, não há causalidade ou dependência.
*  'GarageYrBlt' x 'GarageCars': Ano de construção da garagem e número de carros, apesar de correlacionadas não é uma relação de dependência.
*  'GarageYrBlt' x 'GarageArea': Ano de construção da garagem e área da garagem. Não é uma relação de dependência. 
*  'GarageCars' x 'SalePrice': Relação linear positiva entre a capacidade da garagem (em carros) e a variável target. 
*  'GarageCars' x 'GarageArea': Relação entre a capacidade da garagem em carros e a área da garagem, ambas variáveis tratam da capacidade da garagem e são dependentes.

Agora partindo para a análise das variáveis categóricas:

In [ ]:
for i in df_graficos_cat.columns:
  if i == 'SalePrice':
    continue

  plt.subplots(1,2,figsize=(30,10))

  plt.subplot(1,2,1)
  sns.boxenplot(data=df_graficos_cat, x=i, y='SalePrice', showfliers=False)
  sns.stripplot(data=df_graficos_cat, x=i, y='SalePrice', color=".26", size=3)
  plt.title(f'Análise de {i}')

  plt.subplot(1,2,2)
  sns.histplot(data=df_graficos_cat, x=i)
  plt.title(f'Análise de {i}')

plt.show()

O objetivo dessa análise, é mostrar as principais características das variáveis categóricas deste dataset:
*  As variáveis categóricas, de forma geral, apresentam um valor dominante (A nota ‘5’ em ‘OverallQual’, por exemplo), que possui um perfil de preço bem definido. 
*  Quando analisamos as outras características inseridas em cada variável categórica, percebemos uma grande mudança no preço seja para cima ou para baixo.
As 6 variáveis ilustradas nos gráficos mostram bem como esse efeito acontece:
*  'OverallCond': Essa variável avalia a qualidade geral da casa com uma nota de 1 até 10. Através do histograma, percebemos que há um grande volume de notas ‘5’. Apesar de haver notas ‘5’ em todos os preços, os preços estão mais concentrados na faixa de 100000 até 200000. Quando analisamos outras notas, temos casas mais baratas ou mais caras.
*  'OverallQual': Essa variável trata da qualidade dos materiais utilizados na construção da casa. Uma das poucas variáveis que não possuem um valor dominante, porém, a maioria das observações estão concentradas nas notas ‘5’, ’6’ e ‘7’. Quando analisamos as alterações nos preços da casa junto a mudança na qualidade dos materiais notamos uma queda para qualidade baixa e um aumento acentuado para notas altas.
*  'SaleType': Essa variável aborda o tipo da venda. Percebemos que a grande maioria das vendas foram feitas sobre o contrato ‘WD’ e que estão concentradas na faixa de 100000 até 200000, quando analisamos os outros tipos de venda há grandes flutuações nos preços.
*  'PoolQC': Trata da qualidade da piscina, ela existe somente para casas com piscina e existem pouquíssimas casas com piscina neste dataset. O preço médio das casas com piscina é mais alto do que as casas se piscina. Piscinas de qualidade ‘Ex’ possuem preços muito acima do padrão.
*  'LotShape': Aborda o formato do lote, o valor ‘Reg’ é predominante. Os valores alternativos ‘IR1’, ‘IR2’ e ‘IR3’ possuem valores mais altos que os valores presentes em casas com terrenos regulares.
*  'KitchenQual': Aborda a qualidade da cozinha, a maioria dos valores estão concentrados em ‘Gd’ e ‘TA’. Notamos médias diferentes entre valores de ‘Gd’ (good ou bom) e ‘TA’ (Typical/Average ou Típica/Média) que obedecem às noções ordinais de qualidade. O mesmo acontece com ‘Ex’ (Excellent ou Excelente) com o aumento do preço e ‘Fa’ (Fair ou Razoável) com a queda no preço.  

O mesmo efeito acontece na maioria das variáveis categóricas do dataset.


# 4. Preparação dos dados (1.0 ponto)
Faça um processamento nos dados, preenchendo valores faltantes, removendo dados ou variáveis inconsistentes e normalizado os dados

Devido as analises anteriores, as seguintes colunas serão retiradas por se tratarem de variáveis que possuem algum tipo de dependencia:
['YearBuilt','GarageYrBlt',"TotalBsmtSF', 'GrLivArea','GarageCars']

In [ ]:
df = df.drop(['YearBuilt','GarageYrBlt','TotalBsmtSF', 'GrLivArea','GarageCars'], axis=1)

In [ ]:
print(df.shape,"\n\n")

df_na_cols = df.isna().any(axis = 0)[df.isna().any(axis = 0)].index # Variáveis que possuem algum valor faltante
df_na_cols

(1460, 75) 




Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'],
      dtype='object')

In [ ]:
df_na = df[df_na_cols]

print(df_na.shape,"\n\n")
for i in range(0,len(df_na.columns)):
  print(f"Coluna {df_na.columns[i]}")
  print(f"Nulos: {df_na.iloc[:,i].isnull().sum()}")
  print(f"Nulos: {df_na.iloc[:,i].unique()}\n")
  print("-"*100,"\n")

(1460, 18) 


Coluna LotFrontage
Nulos: 259
Nulos: [ 65.  80.  68.  60.  84.  85.  75.  nan  51.  50.  70.  91.  72.  66.
 101.  57.  44. 110.  98.  47. 108. 112.  74. 115.  61.  48.  33.  52.
 100.  24.  89.  63.  76.  81.  95.  69.  21.  32.  78. 121. 122.  40.
 105.  73.  77.  64.  94.  34.  90.  55.  88.  82.  71. 120. 107.  92.
 134.  62.  86. 141.  97.  54.  41.  79. 174.  99.  67.  83.  43. 103.
  93.  30. 129. 140.  35.  37. 118.  87. 116. 150. 111.  49.  96.  59.
  36.  56. 102.  58.  38. 109. 130.  53. 137.  45. 106. 104.  42.  39.
 144. 114. 128. 149. 313. 168. 182. 138. 160. 152. 124. 153.  46.]

---------------------------------------------------------------------------------------------------- 

Coluna Alley
Nulos: 1369
Nulos: [nan 'Grvl' 'Pave']

---------------------------------------------------------------------------------------------------- 

Coluna MasVnrType
Nulos: 8
Nulos: ['BrkFace' 'None' 'Stone' 'BrkCmn' nan]

--------------------------------------------------

*   'LotFrontage': Variável numérica continua, 259 faltantes, será tratada com a média
*   'Alley': Variável categórica, 1369 faltantes, será deletada
*   'MasVnrType': Variável categórica, 8 faltantes, não há valor NaN previsto, será substituído pela moda.
*   'MasVnrArea': Variável numérica, 8 faltantes, seria tratado pela média, porém, para haver área o tipo precisa ser diferente de 'None', se a moda for 'None' a area será 0.
*   'BsmtQual':  Variável categórica, 37 faltantes, NaN significa que não há porão, será substituído por um novo valor.
*   'BsmtCond': Variável categórica, 37 faltantes, NaN significa que não há porão, será substituído por um novo valor.
* 'BsmtExposure': Variável categórica, 38 fatantes, NaN significa que não há porão, será substituído por um novo valor.
*  'BsmtFinType1': Variável categórica, 37 faltantes, NaN significa que não há porão, será substituído por um novo valor.
*   'BsmtFinType2': Variável categórica, 38 faltantes, NaN significa que não há porão, será substituído por um novo valor.
*   'Electrical': Variável categórica, 1 valor faltante, não há valor NaN previsto, será substituído pela moda.
*   'FireplaceQu': Variável categórica, 690 faltantes, NaN significa que não há piscina, normalmente substituído por um novo valor, porém, há muitos valores faltantes, a melhor decisão é excluir a coluna.	
*   'GarageType': Variável categórica, 81 faltantes, NaN significa que não há garagem, será substituído por um novo valor.
*   'GarageFinish': Variável categórica, 81 faltantes, NaN significa que não há garagem, será substituído por um novo valor.
*   'GarageQual’: Variável categórica, 81 faltantes, NaN significa que não há garagem, será substituído por um novo valor.
*   'GarageCond’: Variável categórica, 81 faltantes, NaN significa que não há garagem, será substituído por um novo valor.
*   'PoolQC’: Variável categórica, 1453 faltantes, NaN significa que não há piscina, normalmente substituído por um novo valor, porém, há muitos valores faltantes, a melhor decisão é excluir a coluna.	
*   'Fence': Variável categórica, 1179 faltantes, NaN significa que não há piscina, normalmente substituído por um novo valor, porém, há muitos valores faltantes, a melhor decisão é excluir a coluna.	
*   'MiscFeature': Variável categórica, 1406 faltantes, NaN significa que não há piscina, normalmente substituído por um novo valor, porém, há muitos valores faltantes, a melhor decisão é excluir a coluna.	

Importante notar que ['MasVnrType', 'MasVnrArea'] são relacionadas, caso 'MasVnrType' seja 'None', 'MasVnrArea' deve ser '0'

In [ ]:
df = df.drop(['Alley','PoolQC','Fence','MiscFeature', 'FireplaceQu'], axis = 1)

In [ ]:
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
df['BsmtQual'] = df['BsmtQual'].fillna('NA')
df['BsmtCond'] = df['BsmtCond'].fillna('NA')
df['BsmtExposure'] = df['BsmtExposure'].fillna('NA')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('NA')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('NA')
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
df['GarageType'] = df['GarageType'].fillna('NA')
df['GarageFinish'] = df['GarageFinish'].fillna('NA')
df['GarageQual'] = df['GarageQual'].fillna('NA')
df['GarageCond'] = df['GarageCond'].fillna('NA')

In [ ]:
# Analise separada de 'MasVnrType' devido a relação com 'MasVnrArea'
print(df['MasVnrType'].mode())

0    None
dtype: object


In [ ]:
# Já que a moda de 'MasVnrType' é 'None', então 'MasVnrArea' deve ser preenchido com 0
df['MasVnrType'] = df['MasVnrType'].fillna(df['MasVnrType'].mode()[0])
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 70 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

Todos os valores faltantes foram preenchidos, os dados que não agregam a análise também foram removidos.  

A normalização dos dados é feita da seguinte forma:

In [ ]:
# Primeiramente precisamos dividir o dataset novamente, as variáveis categóricas precisam passar por um tratamento antes de serem devidamente normalizadas.
# Separação das variáveis em númericas e categóricas
df_categoricas = df.select_dtypes('object')
df_numericas = df.select_dtypes(exclude='object')

# Dentro do dataframe com variáveis numéricas, ainda temos as variáveis categoricas ordinais presentes no dataset
# As variáveis 'MSSubClass', 'OverallQual', 'OverallCond' precisam ser adicionadas ao dataframe de categoricas
df_categoricas = pd.concat([df_categoricas, df_numericas[['MSSubClass', 'OverallQual', 'OverallCond']]], axis = 1)
df_numericas = df_numericas.drop(['MSSubClass', 'OverallQual', 'OverallCond'], axis = 1)
df_categoricas = df_categoricas.astype('str')
df_categoricas = pd.get_dummies(df_categoricas)
df_final = pd.concat([df_categoricas, df_numericas], axis=1)

In [ ]:
X = df_final.drop(['SalePrice'],axis=1)
y = df_final['SalePrice']

Com o dataset bem ajustado, ainda precisamos lidar com as colunas que possuem valores dominantes e que não agregam em nada ao nosso modelo. A função VarianceThreshold() cumpre bem essa função e elimina essas colunas.

In [ ]:
vari_thresh = VarianceThreshold(threshold=0.01)
vari_thresh.fit(X)

cols = vari_thresh.get_support()

In [ ]:
X.iloc[:,cols].columns

Index(['MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM',
       'LotShape_IR1', 'LotShape_IR2', 'LotShape_Reg', 'LandContour_Bnk',
       'LandContour_HLS', 'LandContour_Low',
       ...
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object', length=218)

# 5. Criação e seleção de características (1.0 ponto)
Observando as variáveis disponíveis, é possível criar novas variáveis? Ou trazer de um dado externo novos dados para agregar aos existentes? É possível selecionar uma variável ou grupo de variáveis que melhor explica a variável dependente?

**Observando as variáveis disponíveis, é possível criar novas variáveis?**   

Sim, é possível criar novas variáveis, como vimos anteriormente, algumas variáveis presentes no dataset trata-se de variáveis construídas a partir de outras. Sabendo disso, poderíamos, por exemplo, analisar o custo por pé quadrado das casas ou o custo por nº de salas.  


**Ou trazer de um dado externo novos dados para agregar aos existentes?**  

Sim, mas com diversas etapas antes de serem propriamente usados. Primeiramente, esses dados precisam ser adequados para nosso modelo, isso é, pertencerem ao mesmo perfil da amostra que está sendo estudada. Não adianta trazer dados de casas brasileiras para um estudo de casas canadenses, por exemplo. Depois precisaríamos fazer um estudo em relação aos atributos desse novo conjunto de dados e tentar inferir as informações que usamos em nosso modelo a partir das informações que estão inseridas nesse novo conjunto. Por exemplo, caso o preço da compra de uma venha uma unidade monetária diferente (Dólar ou Euro), essa unidade precisaria ser transformada antes de poder ser utilizada.

**É possível selecionar uma variável ou grupo de variáveis que melhor explica a variável dependente?**  

Sim, é possível definir um grupo de variáveis que melhor explica a variável dependente. Podemos utilizar diversos métodos (RFE, KBest, DecisionTreeRegressor e etc.). As variáveis mais importantes serão selecionadas abaixo através do KBest.


In [ ]:
# Como há muitas variáveis categoricas e estamos utilizando dummies, o dataset possui muitas colunas

print(f'Colunas: {len(X.iloc[:,cols].columns)}')

Colunas: 218


In [ ]:
# Com 218 colunas sendo estudadas, inicialmente, tentarei selecionar as 218/2 = 109 atributos mais importantes

X_new = SelectKBest(f_classif, k=109).fit(X.iloc[:,cols], y).get_feature_names_out(X.iloc[:,cols].columns) # SelectKBest utilizando as colunas selecionadas pelo VarianceThreshold()
X_new

array(['MSZoning_FV', 'LotShape_IR1', 'LotShape_Reg', 'LandContour_HLS',
       'LotConfig_CulDSac', 'LandSlope_Gtl', 'LandSlope_Mod',
       'Neighborhood_Blmngtn', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_IDOTRR',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt',
       'Neighborhood_Somerst', 'Neighborhood_StoneBr',
       'Neighborhood_Timber', 'Condition1_PosN', 'Condition1_RRAn',
       'HouseStyle_2Story', 'RoofStyle_Gable', 'RoofStyle_Hip',
       'Exterior1st_CemntBd', 'Exterior1st_Stucco', 'Exterior1st_VinylSd',
       'Exterior2nd_BrkFace', 'Exterior2nd_CmentBd', 'Exterior2nd_Stucco',
       'Exterior2nd_VinylSd', 'MasVnrType_None', 'MasVnrType_Stone',
       'ExterQual_Ex', 'ExterQual_Gd', 'ExterQual_TA',
       'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab',
       'BsmtQual_Ex', 'BsmtQual_Gd', 'BsmtQual_NA', 'BsmtQual_TA',
       'BsmtCond_Gd', 'BsmtCond_NA', 'BsmtCond_TA', 'BsmtExposure_Av',
       'BsmtExposure_Gd', 'BsmtExpo

# 6. Decisão do modelo (1.0 ponto)
Dada toda a análise realizada nos pontos anteriores, qual modelo melhor se adequa a este problema? E por quê?

De toda a análise, sabemos que devemos prever uma variável númerica continua. Devido a complexidade do problema, a regressão linear simples será descartada. Inicialmente Ridge, Lasso, SGDRegressor, ElasticNet serão testados através do cross_validade para julgar o melhor modelo. Ainda há outros modelos de regressão que podem ser aplicados aqui, como GradientBoostingRegressor, DecisionTreeRegressor, RandomForestRegressor

In [ ]:
X = MinMaxScaler().fit_transform(X.loc[:,X_new]) # X normalizado e com as novas colunas


In [ ]:
rid = Ridge()
las = Lasso(max_iter=10000)
sgd = SGDRegressor()
ela = ElasticNet()
gbr = GradientBoostingRegressor()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()


modelos = [rid, las, sgd, ela, gbr, dtr, rfr]

metric_r2 = []
metric_max = []
metric_RMSE = []
metric_MAE = []
nomes = []

for i in modelos:
  print('-'*20)
  print(f'{i}')
  nomes.append(f'{i}')
  cv = cross_validate(i, X, y, cv=5, scoring=['r2','max_error','neg_mean_absolute_error','neg_root_mean_squared_error'])
  print(f'r2: {cv["test_r2"].mean():.4f} +/- {cv["test_r2"].std():.4f}')
  metric_r2.append(cv["test_r2"].mean())
  print(f'erro máximo: {cv["test_max_error"].mean():.4f} +/- {cv["test_max_error"].std():.4f}')
  metric_max.append(cv["test_max_error"].mean())
  print(f'MAE: {cv["test_neg_mean_absolute_error"].mean():.4f} +/- {cv["test_neg_mean_absolute_error"].std():.4f}')
  metric_MAE.append(cv["test_neg_mean_absolute_error"].mean())
  print(f'RMASE: {cv["test_neg_root_mean_squared_error"].mean():.4f} +/- {cv["test_neg_root_mean_squared_error"].std():.4f}')
  metric_RMSE.append(cv["test_neg_root_mean_squared_error"].mean())
print('-'*20)

--------------------
Ridge()
r2: 0.8378 +/- 0.0568
erro máximo: -280827.7671 +/- 135693.7395
MAE: -19014.9776 +/- 1348.7202
RMASE: -31600.5538 +/- 6526.9988
--------------------
Lasso(max_iter=10000)
r2: 0.8312 +/- 0.0645
erro máximo: -292858.7046 +/- 156137.4522
MAE: -19153.3587 +/- 1352.9010
RMASE: -32175.1485 +/- 7095.6845
--------------------
SGDRegressor()
r2: 0.8374 +/- 0.0467
erro máximo: -263667.5324 +/- 100208.4857
MAE: -19530.6599 +/- 1383.5102
RMASE: -31752.2015 +/- 5756.2598
--------------------
ElasticNet()
r2: 0.6290 +/- 0.0457
erro máximo: -353122.8914 +/- 89451.3063
MAE: -31062.7309 +/- 2460.3351
RMASE: -48287.6644 +/- 6224.5897
--------------------
GradientBoostingRegressor()
r2: 0.8642 +/- 0.0330
erro máximo: -234686.5185 +/- 93695.8380
MAE: -17943.9695 +/- 1666.2004
RMASE: -29057.4993 +/- 4566.7386
--------------------
DecisionTreeRegressor()
r2: 0.6889 +/- 0.0447
erro máximo: -237568.0000 +/- 57440.7895
MAE: -29218.8877 +/- 1747.1563
RMASE: -43877.0985 +/- 3060.4180

Das métricas apresentadas pelo Cross Validate, temos que o melhor modelo é o GradientBoostingRegressor() e este será o modelo adotado para o restante do trabalho.

# 7. Modelagem (1.0 ponto)
Divida os dados em teste e treino, construa o seu modelo e teste alterar os parâmetros utilizados por ele.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 82)

modelo = GradientBoostingRegressor(random_state=35) # Parâmetros default. Random state para replicar resultados.
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print(r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(math.sqrt(mean_squared_error(y_test, y_pred)))
print(max_error(y_test, y_pred))

0.8827359292397841
17214.709590862825
26745.641285656577
211711.6687465537


In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(x=y_test, y=y_test)
sns.regplot(x=y_test, y=y_pred);

Ao invés de testar manualmente diversos parâmetros do modelo, GridSearchCV() será utilizado para testar diversas combinações de parâmetros e então selecionar a melhor.

In [ ]:
# Utilizando o GridSearchCV() para testar os diversos parâmetros do modelo.
parametros = {'learning_rate':np.arange(0.0, 2.1, 0.1),
              'criterion':('friedman_mse', 'squared_error'),
              'min_samples_split':[2,3],
              'max_depth':[3,4,5]}

gscv = GridSearchCV(modelo, parametros, scoring='r2')
gscv.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 1260.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py", line 525, in fit
    self._check_params()
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py", line 275, in _check_params
    "learning_rate must be greater than 0 but was %r" % self.learning_rate
ValueError: learning_rate 

GridSearchCV(estimator=GradientBoostingRegressor(random_state=35),
             param_grid={'criterion': ('friedman_mse', 'squared_error'),
                         'learning_rate': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ]),
                         'max_depth': [3, 4, 5], 'min_samples_split': [2, 3]},
             scoring='r2')

In [ ]:
parametros_otimizados = pd.DataFrame(gscv.cv_results_)[['params','rank_test_score','mean_test_score']].sort_values(by=['rank_test_score']) # Dataframe de resultados
parametros_otimizados

,params,rank_test_score,mean_test_score
136,"{'criterion': 'squared_error', 'learning_rate'...",1,0.839401
10,"{'criterion': 'friedman_mse', 'learning_rate':...",2,0.839123
140,"{'criterion': 'squared_error', 'learning_rate'...",3,0.838231
14,"{'criterion': 'friedman_mse', 'learning_rate':...",4,0.837612
137,"{'criterion': 'squared_error', 'learning_rate'...",5,0.837026
...,...,...,...
3,"{'criterion': 'friedman_mse', 'learning_rate':...",248,NaN
2,"{'criterion': 'friedman_mse', 'learning_rate':...",249,NaN
1,"{'criterion': 'friedman_mse', 'learning_rate':...",250,NaN
131,"{'criterion': 'squared_error', 'learning_rate'...",251,NaN


In [ ]:
gscv.best_params_

{'criterion': 'squared_error',
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_samples_split': 2}

In [ ]:
modelo = GradientBoostingRegressor(criterion = 'squared_error', learning_rate = 0.1, max_depth = 5, min_samples_split = 2, random_state=35) # Parâmetros testados pelo GridSearchCV()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print(r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(math.sqrt(mean_squared_error(y_test, y_pred)))
print(max_error(y_test, y_pred))

0.8799534912830984
17662.214838909164
27061.090853708774
172567.76408412977


In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(x=y_test, y=y_test)
sns.regplot(x=y_test, y=y_pred);

# 8. Histórico (1.0 pontos)
Preencha a tabela abaixo com cada teste realizado com seu modelo. Não se preocupe caso voce mude de ideia sobre qual modelo testar, mas não deixe de testá-lo exaustivamente e, caso precise, retorne para o passo de criação e seleção de características.

Modelo | Parâmetros | Acurácia | Precisão
-------|------------|----------|---------
Modelo 1| abc|abc|abc
Modelo 2|xyz|xyz|xyz


Foram realizadas duas baterias de testes. O primeiro teste tratou da comparação entre diversos modelos utilizando o cross_validate para encontrar o modelo que possui as melhores métricas em situações distintas.

Nesta etapa foram testados os seguintes modelos: Ridge, Lasso, SGDRegressor,
ElasticNet, GradientBoostingRegressor, DecisionTreeRegressor e RandomForestRegressor que tiveram os seguintes resultados:

In [ ]:
print('''As metricas MAE (Erro Médio Absoluto) e RMSE (Erro quadrático médio) estão negativos por que é a forma que o cross_validate nos fornece essa informação através dos argumentos:
*    test_neg_mean_absolute_error
*    test_neg_root_mean_squared_error

''')


pd.DataFrame({'r2':metric_r2,
              'max':metric_max,
              'MAE':metric_MAE,
              'RMASE':metric_RMSE}, index=nomes)

As metricas MAE (Erro Médio Absoluto) e RMSE (Erro quadrático médio) estão negativos por que é a forma que o cross_validate nos fornece essa informação através dos argumentos:
*    test_neg_mean_absolute_error
*    test_neg_root_mean_squared_error




,r2,max,MAE,RMASE
Ridge(),0.837788,-280827.767147,-19014.977593,-31600.553793
Lasso(max_iter=10000),0.831202,-292858.704620,-19153.358673,-32175.148530
SGDRegressor(),0.837358,-263667.532389,-19530.659922,-31752.201487
ElasticNet(),0.628976,-353122.891396,-31062.730877,-48287.664379
GradientBoostingRegressor(),0.864197,-234686.518538,-17943.969472,-29057.499298
DecisionTreeRegressor(),0.688884,-237568.000000,-29218.887671,-43877.098536
RandomForestRegressor(),0.844068,-238494.384000,-18848.933558,-31228.906033


Ao análisar as métricas e concluir que o GradientBoostingRegressor() é o melhor regressor para nossa situação, este modelo foi testado de maneira intensa através da função GridSearchCV().

Esta função permite testar todas as combinações entre os parâmetros fornecidos, os parâmetros testados foram os seguintes:
* 'learning_rate': de 0 até 2 em degraus de 0.1 (0.1, 0.2, 0.3...)
* 'criterion': 'friedman_mse' e  'squared_error'
* 'min_samples_split': 2 e 3
* 'max_depth': 3, 4 e 5

Que resultaram em 252 combinações de parâmetros com métricas distintas. Os resultados estão listados abaixo.

In [ ]:
parametros_otimizados

,params,rank_test_score,mean_test_score
136,"{'criterion': 'squared_error', 'learning_rate'...",1,0.839401
10,"{'criterion': 'friedman_mse', 'learning_rate':...",2,0.839123
140,"{'criterion': 'squared_error', 'learning_rate'...",3,0.838231
14,"{'criterion': 'friedman_mse', 'learning_rate':...",4,0.837612
137,"{'criterion': 'squared_error', 'learning_rate'...",5,0.837026
...,...,...,...
3,"{'criterion': 'friedman_mse', 'learning_rate':...",248,NaN
2,"{'criterion': 'friedman_mse', 'learning_rate':...",249,NaN
1,"{'criterion': 'friedman_mse', 'learning_rate':...",250,NaN
131,"{'criterion': 'squared_error', 'learning_rate'...",251,NaN


Por fim, o modelo foi ajustado com as seguintes caracteristicas: (criterion = 'squared_error', learning_rate = 0.1, max_depth = 5, min_samples_split = 2) e obteve as seguintes métricas:

* R2_score = 0.8799534912830984
* MAE = 17662.214838909164
* RMSE = 27061.090853708774
* Erro máximo = 172567.76408412977


# 9. Resultado (2.0 pontos)
Reúna os seus achados, os valores das métricas do seu melhor modelo obtido e o melhor conjunto de variáveis. Escreva abaixo um relatório que seria enviado para uma diretoria de uma empresa sobre seu trabalho.

Bom dia,  
De acordo com as demandas discutidas nas reuniões anteriores, segue os resultados obtidos da análise das vendas de casas executadas em Ames, Iowa. Este relatório aborda os seguintes pontos:

* Análise exploratória dos dados
* Preparação e seleção de características
* Decisão de modelos e modelagem
* Resultados

## Análise exploratória dos dados

O conjunto de dados consiste em 1460 observações com 80 atributos sendo 79 variáveis independentes e 1 variável dependente. Dos 80 atributos, 46 são categóricos e 33 são quantitativos.

### Atributos quantitativos (numéricos)

Analisamos as correlações entre os atributos numéricos, notamos que grande parte dos atributos possuem uma alta correlação com o a variável target (‘SalePrice’). Porém, alguns atributos possuem uma grande correlação entre si, o que pode ser um indicativo de dependência e causalidade. 

Através da análise dos gráficos, podemos notar relações de dependência nas seguintes variáveis: 'YearBuilt','GarageYrBlt',"TotalBsmtSF', 'GrLivArea' e 'GarageCars'. Chegamos a conclusão que essas variáveis são obtidas pela combinação de variáveis dentro do próprio dataset.

### Atributos qualitativos (categóricos)

Neste ponto notamos que, de forma geral, as casas apresentam características e faixas de preço bem definidos. A grande maioria das casas pertence a faixa de preço de 100000 até 2000000.  

Ao analisar os histogramas das variáveis categóricas, notamos que algumas características são tão bem definidas que possuem um valor dominante em relação aos outros, esse tipo de característica não adiciona nada as técnicas de machine learning. Sendo assim, removemos toda característica que tenha uma variância menor que 0.01, simplificando e reduzindo a dimensão do nosso conjunto de dados.

## Preparação e seleção de características

Observando as informações presentes dentro do dataset, concluímos que há 18 características com valores nulos. O tratamento de valores nulos varia de característica para característica. Em sua grande maioria, a moda foi aplicada para variáveis categóricas e a média para variáveis numéricas. As exceções são:

* ‘PoolQC’, ‘Fence’, ‘MiscFeature’, ‘FireplaceQu’ e ‘Alley’: Variáveis com muitos valores faltantes, não adicionam nada ao estudo então devem ser excluídas.

* 'MasVnrType' e 'MasVnrArea': Como a moda de ‘MasVnrType’ é ‘None’, a variável ‘MasVnrArea’ deve ser preenchida com zeros. 

Por estarmos lidando com um problema de regressão, as variáveis categóricas precisam ser tratadas. O desempenho para sistemas lineares é melhor para variáveis categóricas discretizadas, ou seja, uma coluna binária para cada valor presente na variável. Após este tratamento, o dataset ajustado para a modelagem tem 218 colunas (teria 305 colunas sem todo o tratamento)  
Para tentar diminuir a complexidade do modelo, aplicamos a função SelectKBest() com o estimador f_classif para selecionar as características mais importantes e tentarmos reduzir pela metade o número de variáveis. 
Após isso, os dados foram normalizados de 0 até 1 para que não tenhamos problemas com suas grandezas.
## Decisão de modelos e modelagem
Existem diversos modelos de regressão, e devido a complexidade do modelo, optamos por não utilizar a regressão linear. Utilizamos os modelos de regressão múltipla e outros baseados em árvores utilizando boosting e ensemble. Os modelos testados foram: Ridge, Lasso, SGDRegressor, ElasticNet, GradientBoostingRegressor, DecisionTreeRegressor e RandomForestRegressor com corss_validate() para termos desempenhos com diversas amostras do conjunto de dados.  
A conclusão foi que o modelo mais adequado para o problema é o GradientBoostingRegressor .  
## Resultados
Ao definir o modelo, fizemos uma execução com suas características padrão e tivemos as seguintes métricas: 

* R2: 0.8827359292397841
* MAE: 17214.709590862825
* RMSE: 26745.641285656577
* Erro máximo: 211711.6687465537

Com a função GridSearchCV() fizemos o ajuste fino dos hiper parâmetros da do modelo GradientBoostingRegressor() aumentamos o desempenho do modelo nas seguintes métricas:

* R2: 0.8799534912830984
* MAE: 17662.214838909164
* RMSE: 27061.090853708774
* Erro máximo: 172567.76408412977

Apesar da diminuição de outras métricas, o erro máximo se torna muito mais aceitável no segundo modelo.